# KNN

In [ ]:

from models.knn_classifier import KNNClassifier
knn = KNNClassifier()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../images/tornillos/tornillos0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_vec, category, object_length, images_dict = knn.predict(gray)

print("KNN:", category, object_length)

del images_dict['label_image']
fig = plt.figure(figsize=(15, 5))
for i, img in enumerate(images_dict):
    ax = fig.add_subplot(1, 4, i+1)
    ax.imshow(cv2.cvtColor(images_dict[img], cv2.COLOR_BGR2RGB))
    ax.set_title(img)
    ax.axis('off')

plt.show()

# K Means

In [ ]:
from models.kmeans_classifier import KMeansClassifier
k_means = KMeansClassifier()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../images/tornillos/tornillos0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_vec, final_centroids, category, object_length, images_dict = k_means.predict(gray)

print("K Means:", category, object_length)

del images_dict['label_image']
fig = plt.figure(figsize=(15, 5))
for i, img in enumerate(images_dict):
    ax = fig.add_subplot(1, 4, i+1)
    ax.imshow(cv2.cvtColor(images_dict[img], cv2.COLOR_BGR2RGB))
    ax.set_title(img)
    ax.axis('off')

plt.show()

# Plots Comparativos

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../images/tornillos/tornillos0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_vec0, centroids, category0, object_length0, images_dict0 = k_means.predict(gray)
img_vec, category, object_length, images_dict = knn.predict(gray)

print("KMeans:", category0, object_length0)
print("KNN:", category, object_length)

plotKmeans = k_means.generate_data_plots(img_vec0, centroids)
plotsKnn = knn.generate_data_plots(img_vec)

fig = plt.figure(figsize=(20, 20))
for i, (plot, title) in enumerate(zip([plotsKnn, plotKmeans], ["K Nearest Neighbors", "K Means"])):
    ax = fig.add_subplot(1, 2, i+1)
    plt.imshow(plot["plot_Momento de Hu 1 vs Excentricidad"])
    ax.set_title(title)
    ax.grid(False)
    ax.set_axis_off()

plt.show()